In [ ]:
import pandas as pd








```
# This is formatted as c

```
# This is fo
```

ode
```

# New section

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
import sklearn.linear_model as lm
from sklearn import metrics
#from regressors import stats as regresion_stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression




/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# New section

#Step 1: Regress the i asset/portfolio on the time series risk factor to obtain beta
#Step 2:  Obtain a matrix of beta 636 x 3 
#Step 3: Multiply the beta matrix with TS average returns to achieve beta premium


In [ ]:
path = '/content/GRS 32p.xlsx'
portfolios = pd.read_excel(path, sheet_name = 'portfolio')

ff = pd.read_excel(path, sheet_name = 'ff factors')

q_model = pd.read_excel(path, sheet_name = 'q factor')

In [ ]:
q_model.describe()

,R_MKT,R_ME,R_IA,R_ROE,R_EG
count,636.000000,636.000000,636.000000,636.000000,636.000000
mean,0.532562,0.274202,0.362201,0.538867,0.814053
std,4.471617,3.047915,1.876074,2.494246,1.897952
min,-23.140300,-14.390300,-7.157200,-13.832900,-7.056000
25%,-2.140575,-1.514250,-0.825175,-0.721950,-0.309675
50%,0.908050,0.191900,0.301550,0.641400,0.733150
75%,3.496800,2.026950,1.541025,1.868600,1.899675
max,16.033000,22.136900,9.241100,10.378500,10.815600


In [ ]:
portfolios.describe()

,SMALL LoBM LoOP,ME1 BM1 OP2,ME1 BM1 OP3,SMALL LoBM HiOP,ME1 BM2 OP1,ME1 BM2 OP2,ME1 BM2 OP3,ME1 BM2 OP4,ME1 BM3 OP1,ME1 BM3 OP2,ME1 BM3 OP3,ME1 BM3 OP4,SMALL HiBM LoOP,ME1 BM4 OP2,ME1 BM4 OP3,SMALL HiBM HiOP,BIG LoBM LoOP,ME2 BM1 OP2,ME2 BM1 OP3,BIG LoBM HiOP,ME2 BM2 OP1,ME2 BM2 OP2,ME2 BM2 OP3,ME2 BM2 OP4,ME2 BM3 OP1,ME2 BM3 OP2,ME2 BM3 OP3,ME2 BM3 OP4,BIG HiBM LoOP,ME2 BM4 OP2,ME2 BM4 OP3,BIG HiBM HiOP
count,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000
mean,0.085321,0.604845,0.691407,0.816213,0.659353,0.816889,0.845755,1.017706,0.774828,0.841995,1.030451,1.069865,0.842452,1.015811,1.099368,1.235156,0.363715,0.425203,0.564602,0.575917,0.422536,0.562871,0.631341,0.612414,0.397027,0.610980,0.732267,0.699184,0.556914,0.673892,0.859915,0.688051
std,8.452338,6.967855,6.196712,6.173917,7.191829,5.610353,5.069421,5.653793,6.423315,5.143847,5.036767,5.991571,6.447225,5.743479,6.219712,7.779821,7.882600,5.965213,5.005836,4.644930,5.625397,4.834566,4.501737,4.874470,4.681942,4.373846,4.593142,5.328718,4.942693,4.708038,5.439820,9.746230
min,-36.003700,-33.751900,-32.650600,-31.873300,-34.170100,-29.115600,-28.881500,-27.564100,-30.763100,-31.606800,-21.226300,-23.577800,-31.602300,-27.101200,-26.523100,-32.536400,-32.103600,-24.766300,-21.333900,-23.231100,-21.944300,-24.605200,-24.107800,-26.572100,-26.285900,-18.592700,-20.140900,-20.992700,-25.454400,-23.562600,-22.115500,-44.616000
25%,-4.889500,-3.825900,-3.332950,-2.905025,-3.568075,-2.312800,-1.870725,-1.879275,-2.815775,-2.042850,-1.777400,-2.140200,-2.589925,-1.853000,-2.128225,-2.614675,-4.158250,-3.312125,-2.314450,-2.104000,-2.971900,-2.147450,-1.984125,-2.183575,-2.137925,-1.660600,-1.972550,-2.502175,-2.083950,-1.945650,-1.989800,-3.856525
50%,0.293900,0.987000,0.939050,1.268650,0.875750,1.358950,1.181000,1.505200,0.960900,1.101550,1.404700,1.285150,1.086900,1.319450,1.322050,1.468750,0.574250,0.561300,0.744200,0.822750,0.602300,0.791100,0.804950,0.899650,0.444650,0.810450,0.866300,1.000900,0.855350,0.916000,0.904500,0.652900
75%,5.316750,4.778250,4.769775,4.851850,4.985125,4.483025,3.985525,4.356775,4.393075,4.055625,4.059475,4.598450,4.594075,4.354075,4.822825,5.414175,5.146700,4.084300,3.674650,3.474950,4.137600,3.716300,3.343150,3.589625,3.131675,3.059450,3.430550,4.010575,3.552025,3.457975,4.245375,5.215500
max,46.316900,21.365900,24.967300,25.802800,37.297400,23.254900,23.392100,27.739600,29.001900,27.068100,27.085000,29.006400,31.048800,31.540600,30.286600,42.580900,30.836100,21.258400,24.254700,19.860300,20.666900,23.754100,17.090800,16.231100,15.908100,14.280900,17.033000,20.961700,20.679500,20.337300,21.928300,130.120200


In [ ]:
ff.describe()

,Mkt-RF,SMB,HML,RMW,CMA
count,636.000000,636.000000,636.000000,636.000000,636.000000
mean,0.553318,0.209811,0.307594,0.275550,0.291211
std,4.481373,3.031594,2.868057,2.212315,2.009689
min,-23.240000,-14.860000,-11.060000,-18.480000,-6.860000
25%,-2.057500,-1.550000,-1.365000,-0.810000,-1.015000
50%,0.920000,0.090000,0.255000,0.255000,0.145000
75%,3.505000,2.015000,1.730000,1.305000,1.532500
max,16.100000,18.050000,12.600000,13.380000,9.560000


In [ ]:
data.head()

,SMALL LoBM LoOP,ME1 BM1 OP2,ME1 BM1 OP3,SMALL LoBM HiOP,ME1 BM2 OP1,ME1 BM2 OP2,ME1 BM2 OP3,ME1 BM2 OP4,ME1 BM3 OP1,ME1 BM3 OP2,ME1 BM3 OP3,ME1 BM3 OP4,SMALL HiBM LoOP,ME1 BM4 OP2,ME1 BM4 OP3,SMALL HiBM HiOP,BIG LoBM LoOP,ME2 BM1 OP2,ME2 BM1 OP3,BIG LoBM HiOP,ME2 BM2 OP1,ME2 BM2 OP2,ME2 BM2 OP3,ME2 BM2 OP4,ME2 BM3 OP1,ME2 BM3 OP2,ME2 BM3 OP3,ME2 BM3 OP4,BIG HiBM LoOP,ME2 BM4 OP2,ME2 BM4 OP3,BIG HiBM HiOP
0,17.2862,19.8935,15.6330,18.2804,20.1294,17.3165,11.3451,17.9472,19.6324,14.5552,18.9931,14.3181,15.4490,17.6017,16.8170,11.4224,10.9587,4.6383,5.6178,8.0528,7.3545,3.9653,7.9275,7.3771,5.0590,5.0412,5.9738,11.4081,9.2734,10.5239,15.5762,17.0256
1,6.1325,4.0457,2.2561,3.8413,3.4825,1.2036,2.0576,3.0982,4.8591,-0.7407,2.9962,8.8581,5.0201,0.4644,7.0699,5.4102,0.0616,1.1272,2.0711,2.2794,1.1012,-0.4854,-0.5445,1.6348,-0.2632,-2.2098,0.2988,-2.5594,-1.7836,-1.2639,-0.4976,-3.4834
2,5.5244,5.3522,8.4321,5.6282,7.6959,8.1816,5.6445,6.3539,6.9673,6.7471,2.8268,0.0830,3.3920,6.1220,-1.5143,-3.5198,1.2140,6.8351,2.4438,4.5089,1.3560,1.9851,2.8705,5.7025,2.4372,3.6538,3.0730,9.3053,5.3530,5.1318,9.3183,10.4279
3,3.6228,4.2923,3.6023,5.5480,2.9611,2.7520,4.2744,5.0409,2.1273,3.3518,2.8904,-1.0735,4.2844,5.1733,3.5437,10.5736,12.5540,2.4455,2.6854,7.4278,1.5928,3.4128,3.8140,1.8868,0.0724,0.9495,3.3521,1.7443,3.3673,0.9057,5.5114,-5.0819
4,2.4768,1.1203,-0.7445,-2.7143,-3.1003,-2.3973,-1.7922,-0.8928,-1.2508,-2.3342,-1.3356,-3.3812,-2.0129,-0.6680,-1.4336,-6.4152,-7.8631,-2.7635,-5.6839,-6.0219,-3.0516,-3.7434,-3.9299,-2.6553,-5.0847,-3.4662,-4.4328,-4.1344,-3.4976,-2.1852,-6.0156,-2.9300


In [ ]:
def GRS_1(data, factor):
  if data.shape[0] < factor.shape[0]: factor = factor.loc[data.index]
  else: data = data.loc[factor.index]
  
  data = data.fillna(0)
  factor = factor.fillna(0)

  N = data.shape[1]
  x = np.array(factor)
  x = np.hstack((np.ones((x.shape[0],1)),x))
  alpha = pd.DataFrame(data = np.zeros(shape = (N, 4)),
                      columns = ['alpha' , 't', 'hat', 'mu hat'],
                      index = data.columns)
  R2 = np.zeros(N)
  adj_R2 = np.zeros(N)
  e = np.zeros(shape = data.shape)
  R = pd.DataFrame(np.zeros(N), index= data.columns)
  R_bar = data.mean().mean()
  alpha_hat =  pd.DataFrame(np.zeros(N), index= data.columns)
  i = 0
  for column in data.columns:
    y = np.array(data[column])
    R.loc[column] = y.mean()
    model_1 = sm.OLS(y, x)
    res_1 = model_1.fit()
    #print(res_1.params[0])
    alpha['alpha'].loc[column] = res_1.params[0]
    #alpha['t'].loc[column] = res_1.tvalues[0]
    if abs(res_1.tvalues[0]) > 1.95: alpha['t'].loc[column]  = 1
    else: alpha['t'].loc[column]  = 0
    alpha['hat'].loc[column] = res_1.params[0] * 2 - res_1.bse[0] * 2
    R2[i] = res_1.rsquared
    adj_R2[i] = 1 - (1 - res_1.rsquared) * ( (x.shape[0] - 1) / (x.shape[0] - N - 1) )
    e[:, i] = res_1.resid
    i += 1

  alpha['mu hat'] = (R - R_bar) * 2  - (R - R_bar).std() / np.sqrt(len(R)) * 2
  #print(alpha)
  alpha_1 = np.array(alpha['alpha'])
  average_absolute_alpha = alpha['alpha'].abs().mean()
  average_absolute_alpha_R_i = average_absolute_alpha / (R - R_bar).abs().mean()
  ave_alpha_mu =  alpha['hat'].mean() / alpha['mu hat'].mean()

  F = np.delete(x, 0,1)
  F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))

  T = x.shape[0]
  L = F.shape[1]

  Sigma = e.T.dot(e) / (T - L - 1)
  mu = np.array(factor.mean()).reshape(-1,1)


  Omega = (F - F_bar).T.dot(F - F_bar) / (T - 1)
  GRS = (T/N)( (T - N - L) / (T - L - 1))( alpha_1.T.dot(np.linalg. inv(Sigma)).dot(alpha_1) / (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)) )
  alpha['t'] 
  print(alpha_1)
  print(( alpha_1.T.dot(np.linalg. inv(Sigma)).dot(alpha_1))) 
  return GRS.item(), alpha['t'].sum(), R2.mean(), adj_R2.mean()

In [ ]:
data = portfolios

factor = q_model

def GRS_1(data, factor):
  if data.shape[0] < factor.shape[0]: factor = factor.loc[data.index]
  else: data = data.loc[factor.index]

  data = data.fillna(0)
  factor = factor.fillna(0)

  N = data.shape[1]
  x = np.array(factor)
  x = np.hstack((np.ones((x.shape[0],1)),x))
  alpha = pd.DataFrame(data = np.zeros(shape = (N, 5)),
                      columns = ['alpha' , 't', 'tvalues1', 'hat', 'mu hat'],
                      index = data.columns)
  R2 = np.zeros(N)
  adj_R2 = np.zeros(N)
  e = np.zeros(shape = data.shape)
  R = pd.DataFrame(np.zeros(N), index= data.columns)
  R_bar = data.mean().mean()
  alpha_hat =  pd.DataFrame(np.zeros(N), index= data.columns)
  i = 0
  for column in data.columns:
    y = np.array(data[column])
    R.loc[column] = y.mean()
    model_1 = sm.OLS(y, x)
    res_1 = model_1.fit()
    #print(res_1.params[0])
    alpha['alpha'].loc[column] = res_1.params[0]
    #print(res_1.tvalues[0])
    alpha['tvalues1'].loc[column] = res_1.tvalues[0]
    if abs(res_1.tvalues[0]) > 1.95: alpha['t'].loc[column]  = 1
    else: alpha['t'].loc[column]  = 0
    alpha['hat'].loc[column] = res_1.params[0] * 2 - res_1.bse[0] * 2
    R2[i] = res_1.rsquared
    adj_R2[i] = 1 - (1 - res_1.rsquared) * ( (x.shape[0] - 1) / (x.shape[0] - N - 1) )
    e[:, i] = res_1.resid
    i += 1

  alpha['mu hat'] = (R - R_bar) * 2  - (R - R_bar).std() / np.sqrt(len(R)) * 2
  #print(alpha)
  alpha_1 = np.array(alpha['alpha'])
  average_absolute_alpha = alpha['alpha'].abs().mean()
  average_absolute_alpha_R_i = average_absolute_alpha / (R - R_bar).abs().mean()
  ave_alpha_mu =  alpha['hat'].mean() / alpha['mu hat'].mean()

  F = np.delete(x, 0,1)
  F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))

  T = x.shape[0]
  L = F.shape[1]

  Sigma = e.T.dot(e) / (T - L - 1)
  mu = np.array(factor.mean()).reshape(-1,1)

  #print(alpha_1)
  #print(( alpha_1.T.dot(np.linalg. inv(Sigma)).dot(alpha_1))) 
  #print(Sigma)


  Omega = (F - F_bar).T.dot(F - F_bar) / (T - 1)
  GRS = (T/N) * ( (T - N - L) / (T - L - 1)) * ( alpha_1.T.dot(np.linalg. inv(Sigma)).dot(alpha_1) / (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)).item() )


  #print(GRS)
  #print(GRS)
  return GRS, alpha['alpha'] , alpha['tvalues1']


In [ ]:
GRS_1(portfolios, q_model)

(1.5863315835427025, SMALL LoBM LoOP   -0.207476
 ME1 BM1 OP2       -0.045257
 ME1 BM1 OP3       -0.053083
 SMALL LoBM HiOP   -0.016841
 ME1 BM2 OP1        0.018905
 ME1 BM2 OP2        0.023452
 ME1 BM2 OP3       -0.037703
 ME1 BM2 OP4        0.113346
 ME1 BM3 OP1       -0.083021
 ME1 BM3 OP2        0.023133
 ME1 BM3 OP3        0.170612
 ME1 BM3 OP4        0.130562
 SMALL HiBM LoOP   -0.012041
 ME1 BM4 OP2        0.095568
 ME1 BM4 OP3        0.176864
 SMALL HiBM HiOP    0.417966
 BIG LoBM LoOP      0.445058
 ME2 BM1 OP2        0.017272
 ME2 BM1 OP3        0.184191
 BIG LoBM HiOP     -0.058087
 ME2 BM2 OP1       -0.031593
 ME2 BM2 OP2       -0.149157
 ME2 BM2 OP3       -0.108797
 ME2 BM2 OP4       -0.131829
 ME2 BM3 OP1       -0.178394
 ME2 BM3 OP2       -0.091853
 ME2 BM3 OP3        0.035150
 ME2 BM3 OP4        0.003722
 BIG HiBM LoOP     -0.179239
 ME2 BM4 OP2       -0.137268
 ME2 BM4 OP3        0.058500
 BIG HiBM HiOP      0.014870
 Name: alpha, dtype: float64, SMALL LoBM LoOP   -1.4

In [ ]:
for i in range(32):
    y= (data.iloc[:,i])
    x=data[['Mkt-RF','SMB','HML','RMW','CMA']]
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()
    results.params
    results.tvalues
    print(results.summary())
          

KeyError: ignored

In [ ]:


#OneFactor TEST
y = data['ME1 BM4']
x=data['HML']
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params
results.tvalues
print(results.summary())

#MultiFactor FAMA -5

y = data['ME1 BM2']
x=data[['Mkt-RF','SMB','HML','RMW','CMA']]               
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params
results.tvalues
R2 = print(results.rsquared)
print(results.summary())

data.head()

for col in data:
    
y = data['ME1 BM2']
x=data[['Mkt-RF','SMB','HML','RMW','CMA']]               
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params
results.tvalues
print(results.summary())
print(results.resid)

#Looping accross column 25 portfolios
    
for i in range(32):
    y= (data.iloc[:,i])
    x=data[['Mkt-RF','SMB','HML','RMW','CMA']]
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()
    results.params
    results.tvalues
    print(results.summary())
          
#step1 regression

for i in range(21,25):
    y= (data.iloc[:,i])
    x=data[['Mkt-RF','SMB','HML','RMW','CMA']]
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()
    results.params
    results.tvalues
    print(results.summary())
    R2 = results.rsquared
    e  = results.resid
    T = x.shape[0]

for i in range(1,25):
    print(data.iloc[:,i])

#step2:intercept vector
    
    alpha = results.params[0]
    F = data[['Mkt-RF','SMB','HML','RMW','CMA']]
    L = F.shape[1]
    alpha = np.zeros(shape =(25,1))
    alpha[i] = results.params[0]
    
#step3: residual matrix
    
    e = np.zeros(shape = (636,25))
    e[:, i] = results.resid
    
F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))
Sigma = e.T.dot(e) / (T - L - 1)

F = data[['Mkt-RF','SMB','HML','RMW','CMA']]
F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))
mu = np.array(F.mean()).reshape(-1,1)  

Omega = (F - F_bar).T.dot(F - F_bar) / (T - 1)
N = x.shape[0]
GRS = (T/N)( (T - N - L) / (T - L - 1))( alpha.T.dot(np.linalg. inv(Sigma)).dot(alpha) / (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)) )

GRS = (T/N)*( (T - N - L) / (T - L - 1))*

( alpha.T.dot(np.linalg. inv(Sigma)).dot(alpha) 

/ 
 (1 + mu.T.*(np.linalg.inv(Omega))@dot(mu)) )

#getting singular matrix. why? due to the perfect correlation issues
GRS = (T/N)( (T - N - L) / (T - L - 1))

alpha.T
(np.linalg. inv(Sigma))
alpha
 
 (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)) 

Sigma.corr()

-----------------------------------------------------------------------
#Q FACTOR MODEL Q FACTOR MODEL Q FACTOR MODEL


#q FACTOR MODEL

y = data['ME1 BM2']
x=data[['Mkt-RF','SMB','HML','RMW','CMA']]               
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params
results.tvalues
R2 = print(results.rsquared)
print(results.summary())

data.head()

for col in data:
    
y = data['ME1 BM2']
x=data[['R_MKT','R_ME','R_IA','R_ROE','R_EG']]               
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params
results.tvalues
print(results.summary())
print(results.resid)

#Looping accross column 25 portfolios
    
for i in range(21,25):
    y= (data.iloc[:,i])
    x=data[['R_MKT','R_ME','R_IA','R_ROE','R_EG']]
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()
    results.params
    results.tvalues
    print(results.summary())
          
#step1 regression

for i in range(21,25):
    y= (data.iloc[:,i])
    x=data[['Mkt-RF','SMB','HML','RMW','CMA']]
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()
    results.params
    results.tvalues
    print(results.summary())
    R2 = results.rsquared
    e  = results.resid
    T = x.shape[0]

for i in range(1,25):
    print(data.iloc[:,i])

#step2:intercept vector
    
    alpha = results.params[0]
    F = data[['Mkt-RF','SMB','HML','RMW','CMA']]
    L = F.shape[1]
    alpha = np.zeros(shape =(25,1))
    alpha[i] = results.params[0]
    
#step3: residual matrix
    
    e = np.zeros(shape = (636,25))
    e[:, i] = results.resid
    
F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))
Sigma = e.T.dot(e) / (T - L - 1)

F = data[['Mkt-RF','SMB','HML','RMW','CMA']]
F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))
mu = np.array(F.mean()).reshape(-1,1)  

Omega = (F - F_bar).T.dot(F - F_bar) / (T - 1)
N = x.shape[0]
GRS = (T/N)( (T - N - L) / (T - L - 1))( alpha.T.dot(np.linalg. inv(Sigma)).dot(alpha) / (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)) )

GRS = (T/N)*( (T - N - L) / (T - L - 1))*

( alpha.T.dot(np.linalg. inv(Sigma)).dot(alpha) 

/ 
 (1 + mu.T.*(np.linalg.inv(Omega))@dot(mu)) )
























i = 0
  for column in data.columns:
    y = np.array(data[column])
    F = np.delete(x,0,1)
    model_1 = sm.OLS(y, x)
    res_1 = model_1.fit()
    
    alpha['alpha'].loc[column] = res_1.params[0]
    R2[i] = res_1.rsquared 
    e[:, i] = res_1.resid
    i += 1
    x.shape
  alpha_1 = np.array(alpha['alpha'])
 
  T = x.shape[0]
  L = F.shape[1]
  
  F = np.delete(x, 0,1)
  F_bar = np.tile(F.mean(axis=0), (F.shape[0], 1))
  
  Sigma = e.T.dot(e) / (T - L - 1)
  mu = np.array(factor.mean()).reshape(-1,1)


  Omega = (F - F_bar).T.dot(F - F_bar) / (T - 1)
  GRS = (T/N)( (T - N - L) / (T - L - 1))( alpha_1.T.dot(np.linalg. inv(Sigma)).dot(alpha_1) / (1 + mu.T.dot(np.linalg.inv(Omega)).dot(mu)) )




IndentationError: ignored